In [1]:
import findspark

In [2]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('rec').getOrCreate()

23/07/12 00:28:27 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/12 00:28:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/12 00:28:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.ml.recommendation import ALS

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator

In [8]:
data=spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [9]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [16]:
training,test=data.randomSplit([0.8,0.2])

In [17]:
als=ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [21]:
model=als.fit(training)

In [22]:
predictions=model.transform(test)

In [23]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      6|   1.0|    28|  1.8917016|
|      0|   1.0|    27| 0.43938828|
|      1|   1.0|    26| 0.85855955|
|      4|   2.0|     1|  1.2518893|
|      0|   1.0|    13|  0.9781201|
|      3|   1.0|    13|  1.4580214|
|      5|   1.0|     6| 0.33700514|
|      0|   1.0|     3| 0.07270226|
|      0|   1.0|    20| 0.67686236|
|      0|   1.0|     5|  1.5191786|
|      0|   1.0|    19|  0.7329618|
|      2|   1.0|    19|  2.5625691|
|      4|   1.0|    19|  0.5029945|
|      3|   1.0|    17|-0.19584373|
|      0|   1.0|    23|  0.9343408|
|      6|   2.0|    23| 0.10089281|
|      1|   1.0|     7|  0.7057452|
|      2|   1.0|    25|  1.7025363|
|      1|   1.0|    14|  1.2250519|
|      3|   3.0|    14|  1.2798682|
+-------+------+------+-----------+
only showing top 20 rows



In [24]:
evaluator=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [25]:
rmse=evaluator.evaluate(predictions)

In [26]:
print('RMSE')
print(rmse)

RMSE
1.8282495006876636


In [30]:
single_user=test.filter(test['userId']==11).select(['movieId','userId'])

In [31]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     18|    11|
|     36|    11|
|     39|    11|
|     50|    11|
|     51|    11|
|     59|    11|
|     72|    11|
|     75|    11|
|     77|    11|
|     80|    11|
|     81|    11|
+-------+------+



In [33]:
recommendations=model.transform(single_user)

In [34]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     81|    11|  2.6698842|
|     39|    11|  2.6033752|
|     18|    11|   2.004713|
|     11|    11|  1.3864818|
|     51|    11|  1.1919439|
|     80|    11|  1.0925298|
|     50|    11| 0.14352727|
|     36|    11| 0.04638353|
|     59|    11|-0.35040396|
|     72|    11| -1.4777322|
|     75|    11|  -1.490259|
|     77|    11| -2.3404617|
+-------+------+-----------+

